## Configuração inicial

In [ ]:
import pickle
import pandas as pd

from Utils import templates, utils

In [ ]:
## Configurações base

wandb_key = "b9d02b504b9d57ae7801f351e79da0162387f010"

config = {
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 12,     # número de usuários para filtragem colaborativa
    "nci" :19,      # número de itens para filtragem colaborativa
    "lenlimit" :8,  # limite de tamanho para a lista filmes assistidos 
    "test_run" : 500, # define a quantidade de recomendações, '0' para todos
    "obs": "Testing RunTimes"
}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_1
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K.json")
print(f'Quantidade de Usuários: {len(dataset)}')

---------   
## Escolha do modelo

### gemma-3-4b-it

In [ ]:
config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

### meta-llama-3.1-8b-instruct

In [ ]:
config.update({
    "model_name" :"meta-llama-3.1-8b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

### llama-3.2-3b-instruct

In [ ]:
config.update({
    "model_name" :"llama-3.2-3b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q8_0",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

## Execução

In [ ]:
result_pkl = utils.recommendation_workflow(config,wandb_key,dataset,prompt_template,prompt_format)

## Resultados

In [ ]:
#arq_name = 'ml_100k-zs-nir-su12-ci19-llama-3.2-3b-instruct.pkl'

with open(f'{result_pkl}', 'rb') as f:
    data = pickle.load(f)

#data

In [ ]:
results = []
for key, value in data.items():
    if isinstance(key, int) and isinstance(value, dict):  # Pegando apenas os experimentos
        results.append({
            'Candidates': value.get('candidate_set', ''),
            'Ground Truth': value.get('ground_truth', ''),
            'gt_in_candidate_set': value.get('gt_in_candidate_set', ''),
            #'Input 1': value.get('input_1', ''),
            #'Predictions 1': value.get('predictions_1', ''),
            #'Input 2': value.get('input_2', ''),
            #'Predictions 2': value.get('predictions_2', ''),
            'Input 3': value.get('input_3', ''),
            'Predictions 3': value.get('predictions_3', ''),
            'Recommendations': value.get('recommendations', ''),
            'Hit': value.get('hit', ''),
            'Precision': value.get('precision', ''),
            'Recall': value.get('recall', ''),
            'NDCG': value.get('ndcg', '')
        })

df_results = pd.DataFrame(results)

df_results


In [ ]:
df_results['Predictions 3'][0]